<a href="https://colab.research.google.com/github/zachlim98/w251-submissions/blob/main/hw05/lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs this package locally so we can import it like any other module (see below).
!pip install -e ..

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///workspace/data/ucb_mids_w251_project
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Attempting uninstall: mids-plane-classification
    Found existing installation: mids-plane-classification 0.0.1
    Uninstalling mids-plane-classification-0.0.1:
      Successfully uninstalled mids-plane-classification-0.0.1
  Running setup.py develop for mids-plane-classification


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mids_plane_classification.loaders.dataloader import PlaneDataModule

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torch.utils.data.distributed
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [4]:
# Import the data and set it up. If this is the first time running this, please use
# dm.prepare_data()
dm = PlaneDataModule(train_batch_size=32, val_batch_size=32, data_dir='../data', seed=2)
dm.setup()

Downloading: 100% [470582946 / 470582946] bytes

In [7]:
import wandb

wandb.login()
wandb.init(project="W251-Draft-Models", entity="251-summer-22")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zachlim (251-summer-22). Use `wandb login --relogin` to force relogin


In [8]:
GPU=0

In [9]:
SEED=1

In [10]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [11]:
torch.cuda.device_count()

1

### Set the architecture to resnet 18 below

In [12]:
##########################
ARCH = 'resnet18' # set the architecture to RESNET 18
# please look up how to do that
########################
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=32
VAL_BATCH=32
WORKERS=2
START_EPOCH=0

#config for wandb
wandb.config = {
  "learning_rate": LR,
  "epochs": EPOCHS,
  "batch_size": TRAIN_BATCH,
    "arch": ARCH
}

### Check if cuda is available here

In [13]:
# set your active device to your GPU in this cell
torch.cuda.set_device(GPU)

In [14]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [15]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [16]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    return top1.avg

### Save the checkpoint

In [17]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [19]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [20]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [21]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [22]:
# IMG_SIZE = 224
NUM_CLASSES = 11

### Initialize the model using the architecture you selected above

In [23]:
# select the model
model = models.__dict__[ARCH]()
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Instantiate the loss to cross entropy

In [24]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [25]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [26]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

### Create the train and val dataloaders

In [27]:
# Access the data loaders directly from the modules.
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

In [28]:
best_acc1 = 0

In [30]:
%%time
for epoch in range(START_EPOCH, 30):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()[0]))

Epoch: [0][  0/161]	Time  0.521 ( 0.521)	Data  0.431 ( 0.431)	Loss 2.3440e+00 (2.3440e+00)	Acc@1  15.62 ( 15.62)	Acc@5  62.50 ( 62.50)
Epoch: [0][ 50/161]	Time  0.064 ( 0.073)	Data  0.006 ( 0.014)	Loss 2.0654e+00 (2.2350e+00)	Acc@1  37.50 ( 21.26)	Acc@5  59.38 ( 59.93)
Epoch: [0][100/161]	Time  0.064 ( 0.068)	Data  0.007 ( 0.010)	Loss 2.4998e+00 (2.2529e+00)	Acc@1   9.38 ( 19.80)	Acc@5  50.00 ( 60.02)
Epoch: [0][150/161]	Time  0.063 ( 0.067)	Data  0.006 ( 0.009)	Loss 2.1978e+00 (2.2502e+00)	Acc@1  25.00 ( 19.89)	Acc@5  65.62 ( 59.98)
Test: [ 0/41]	Time  0.454 ( 0.454)	Loss 2.3724e+00 (2.3724e+00)	Acc@1  15.62 ( 15.62)	Acc@5  43.75 ( 43.75)
 * Acc@1 18.033 Acc@5 57.611
lr: 0.049999999999999996
Epoch: [1][  0/161]	Time  0.470 ( 0.470)	Data  0.400 ( 0.400)	Loss 2.0798e+00 (2.0798e+00)	Acc@1  18.75 ( 18.75)	Acc@5  75.00 ( 75.00)
Epoch: [1][ 50/161]	Time  0.063 ( 0.072)	Data  0.006 ( 0.014)	Loss 2.1295e+00 (2.2538e+00)	Acc@1  25.00 ( 19.42)	Acc@5  81.25 ( 60.85)
Epoch: [1][100/161]	Time  0.

In [31]:
wandb.finish()

Loss/train,██▇█▇▇▆▅▆▅▆▆▅▅▅▅▃▃▃▃▃▅▂▂▂▂▂▁▁▁▃▂▁▁▁▁▂▁▁▁
Loss/val,▆▆▆▆▆▆▅▅▆▄▄▅▆▃▃▄█▃▂▄▃▂▁▂▄▁▁▁▄▂▁▁
acc1/train,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▆▆▅▅▇▇▇▇▇███▆▇████▇███
acc1/val,▁▁▁▂▂▃▃▃▂▄▄▃▃▆▆▅▁▆▇▅▆▇█▇▆███▆▇██
acc5/train,▁▂▂▃▃▄▅▅▅▅▆▆▆▇▇▇▇▇██████████████████████
acc5/val,▁▂▂▃▄▄▅▅▃▆▆▅▆██▇▁██▇████▇███████
Loss/train,0.03374
Loss/val,0.25562
acc1/train,99.82436
acc1/val,92.81811
acc5/train,100.0
